In [2]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
path = "/Users/jiahuiguo/Documents/GitHub/Thinkful/Thinkful_Practice/Bootcamp/Data/ny_crime_2014.xls"

df = pd.DataFrame(pd.read_excel(path, skiprows = 4))
df.head()

,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3
0,Adams Village,1851.0,0.0,0.0,NaN,0.0,0.0,0.0,11.0,1.0,10.0,0.0,0.0
1,Addison Town and Village,2568.0,2.0,0.0,NaN,0.0,1.0,1.0,49.0,1.0,47.0,1.0,0.0
2,Afton Village4,820.0,0.0,0.0,0.0,NaN,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,Akron Village,2842.0,1.0,0.0,NaN,0.0,0.0,1.0,17.0,0.0,17.0,0.0,0.0
4,Albany4,98595.0,802.0,8.0,54.0,NaN,237.0,503.0,3888.0,683.0,3083.0,122.0,12.0


In [4]:
df_use = df.rename(columns={ df.columns[2]: "violent_crime",df.columns[3]: "murder",
                    df.columns[4]: "rape",df.columns[5]: "rape2",
                    df.columns[7]: "A_assault",df.columns[8]: "property_crime",
                    df.columns[10]: "larceny_theft",df.columns[11]: "motor_theft"})
df_use = pd.DataFrame(df_use)
df_use.head()

,City,Population,violent_crime,murder,rape,rape2,Robbery,A_assault,property_crime,Burglary,larceny_theft,motor_theft,Arson3
0,Adams Village,1851.0,0.0,0.0,NaN,0.0,0.0,0.0,11.0,1.0,10.0,0.0,0.0
1,Addison Town and Village,2568.0,2.0,0.0,NaN,0.0,1.0,1.0,49.0,1.0,47.0,1.0,0.0
2,Afton Village4,820.0,0.0,0.0,0.0,NaN,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,Akron Village,2842.0,1.0,0.0,NaN,0.0,0.0,1.0,17.0,0.0,17.0,0.0,0.0
4,Albany4,98595.0,802.0,8.0,54.0,NaN,237.0,503.0,3888.0,683.0,3083.0,122.0,12.0


In [5]:
columns_to_drop = {"City", "violent_crime", "rape", "rape2", "A_assault"}

df_use.drop(columns_to_drop, axis=1, inplace = True)

df_use.fillna(0, inplace = True)
columns_to_cat = { "murder",  "Robbery", "property_crime", "Burglary", "larceny_theft", "motor_theft", "Arson3"}

for c in columns_to_cat:
    df_use[c+"_cat"] = df_use[c].apply(lambda x: 1 if x>0 else 0) 

df_use.head()

,Population,murder,Robbery,property_crime,Burglary,larceny_theft,motor_theft,Arson3,Burglary_cat,larceny_theft_cat,murder_cat,Arson3_cat,motor_theft_cat,Robbery_cat,property_crime_cat
0,1851.0,0.0,0.0,11.0,1.0,10.0,0.0,0.0,1,1,0,0,0,0,1
1,2568.0,0.0,1.0,49.0,1.0,47.0,1.0,0.0,1,1,0,0,1,1,1
2,820.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0,1,0,0,0,0,1
3,2842.0,0.0,0.0,17.0,0.0,17.0,0.0,0.0,0,1,0,0,0,0,1
4,98595.0,8.0,237.0,3888.0,683.0,3083.0,122.0,12.0,1,1,1,1,1,1,1


In [24]:
df_use["murder"].apply(lambda x: 1 if x>0 else 0) 

0      0
1      0
2      0
3      0
4      1
5      0
6      0
7      1
8      0
9      1
10     0
11     0
12     0
13     0
14     0
15     0
16     0
17     0
18     0
19     0
20     0
21     0
22     0
23     0
24     0
25     0
26     1
27     0
28     0
29     0
      ..
346    0
347    0
348    0
349    0
350    0
351    0
352    1
353    0
354    0
355    0
356    0
357    0
358    0
359    1
360    0
361    0
362    0
363    0
364    0
365    0
366    1
367    0
368    0
369    0
370    0
371    0
372    0
373    0
374    0
375    0
Name: murder, Length: 376, dtype: int64

In [23]:
type(df_use)

pandas.core.frame.DataFrame

In [6]:
#vanilla regression
from sklearn.linear_model import LogisticRegression as lgr

lr = lgr(C=1e9)
y = df_use["Burglary_cat"]
X = df_use.drop("Burglary_cat", axis = 1)

fit = lr.fit(X,y)

# Display.
print('Coefficients')
print(fit.coef_)
print(fit.intercept_)
pred_y_sklearn = lr.predict(X)

print('\n Accuracy by admission status')
print(pd.crosstab(pred_y_sklearn, y))

print('\n Percentage accuracy')
print(lr.score(X, y))

Coefficients
[[-0.00010784  0.00048867  0.0012999   0.08359013  0.06069334  0.02937203
  -0.00398131  0.00047036 -0.00205517  0.00048842  0.00052616 -0.00145105
   0.00087157 -0.00022904]]
[-0.00582057]

 Accuracy by admission status
Burglary_cat   0    1
row_0                
0             18    6
1             25  327

 Percentage accuracy
0.9175531914893617


In [9]:
from sklearn.model_selection import cross_val_score
cross_val_score(lr, X, y, cv=5)

array([0.89473684, 0.88157895, 0.89473684, 0.98648649, 0.97297297])